In [4]:
import tensorflow as tf

import os
# disables all tensorflow logs/errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


tf.Variable(tf.random_uniform([3,1], -1.0,1.0), name = 'var_theta')

<tf.Variable 'var_theta_2:0' shape=(3, 1) dtype=float32_ref>

In [7]:
import numpy as np
val_x = np.array([1,2,3,4])
tf.Variable(val_x,dtype = tf.float32, name = 'var')

<tf.Variable 'var_1:0' shape=(4,) dtype=float32_ref>

In [ ]:
# to assign a variable to a value using tensorlfow use
# w.assign(w+1.0)
# w.assign_add(1.0)

In [ ]:
x = tf.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.Session() as sess:
  print(sess.run(y))  # ERROR: will fail because x was not fed.

  rand_array = np.random.rand(1024, 1024)
  print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.


Important: This tensor will produce an error if evaluated. Its value
must be fed using the feed_dict optional argument to Session.run(), 
Tensor.eval(), or Operation.run().

In [10]:
A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5

with tf.Session() as sess:
    b_val_1 = B.eval(feed_dict = {A: [[1,2,3]]})
    b_val_2 = B.eval(feed_dict = {A:[[4,5,6],[7,8,9]]})
print(b_val_1)

[[6. 7. 8.]]


In [11]:
print(b_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [16]:

n = 10
m = 400
x = tf.placeholder(tf.float32, shape = (None, n+1) , name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [17]:
n_batches

4

In [25]:
init = tf.global_variables_initializer()
n_epochs = 10
learning_rate = 0.01

theta = tf.Variable(tf.random_uniform([n+1, 1], -1,0, 1.0), name = 'theta')
y_pred = tf.matmul(x, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2/m * tf.matmul(tf.transpose(x), error)

training_op = tf.assign(theta, theta - learning_rate * gradients)


def fetch_batch(epoch, batch_index, batch_size):
    # no of epoch
    n_epoch = epoch
    
    # load data from disk
    return x_batch, y_batch


with tf.Session() as sess:
    sess.run(init)
    
    #for epoch in range(n_epochs):
    #    for batch_index in range(n_batches):
            #x_batch,y_batch = fetch_batch(epoch, batch_index, batch_size)
            #sess.run(training_op, feed_dict = {X:x_batch, y: y_batch})
            #best_theta = theta.eval()
            
            
    saver = tf.train.Saver()
    #save_path = saver.save(sess, '/desktop/new_model.ckpt')
    
    
    # to restore model
    #saver.restore(sess, '/desktop/new_model.ckpt')
    
    # to be specific on what you want to save as your model
    # saver = tf.train.Saver({'weights': theta})
    
            
            
            

In [27]:
# logging models with dates at the very beginning of our code
# keeping records with time


from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}'.format(root_logdir, now)


# at the construction phase of the code
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [28]:
# to group a graph with a particular node to a name we have

with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
    
print(error.op.name, mse.op.name)
# the name of each operation in the scope is then prefixed with 'loss'

loss/sub loss/mse


In [29]:
def relu(x,w, b):
    # returns the maximum value between the computed hypothesis and 0
    # so we can't have a negative number as our output
    return max(np.dot(x,w) + b, 0)

In [31]:
#suppose we want to create a graph that adds the output of two rectified
# linear units(RelU)
# here is the implmentation


n_features = 3
X = tf.placeholder(tf.float32 , shape = (None, n_features), name = 'X')

w1 = tf.Variable(tf.random_normal((n_features, 1)), name = 'weights1')
w2 = tf.Variable(tf.random_normal((n_features,1)), name = 'weights2')
b1 = tf.Variable(0.0, name = 'bias1')
b2 = tf.Variable(0.0, name = 'bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name = 'z1')
z2 = tf.add(tf.matmul(X, w2), b2, name= 'z2')

relu1 = tf.maximum(z1, 0., name = 'relu1')
relu2 = tf.maximum(z1, 0., name = 'relu2')
output = tf.add(relu1, relu2, name= 'ouput')

# to add multiple values in place of sum use tf.add_n